In [21]:
import json
import os

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('SA2_2016_AUST.csv')

In [24]:
columns = ['SA4_CODE_2016', 'SA4_NAME_2016', 'GCCSA_NAME_2016', 'STATE_NAME_2016']

In [25]:
mappings_df = df[columns].drop_duplicates().set_index('SA4_CODE_2016')
mappings_df.head()

,SA4_NAME_2016,GCCSA_NAME_2016,STATE_NAME_2016
SA4_CODE_2016,,,
101,Capital Region,Rest of NSW,New South Wales
102,Central Coast,Greater Sydney,New South Wales
103,Central West,Rest of NSW,New South Wales
104,Coffs Harbour - Grafton,Rest of NSW,New South Wales
105,Far West and Orana,Rest of NSW,New South Wales


In [26]:
mappings_df.loc[101,['GCCSA_NAME_2016', 'STATE_NAME_2016']]

GCCSA_NAME_2016        Rest of NSW
STATE_NAME_2016    New South Wales
Name: 101, dtype: object

In [27]:
mappings_df.loc[101]['GCCSA_NAME_2016']

'Rest of NSW'

In [28]:
with open('/Users/hima95/Downloads/Output__spatialise-dataset_Apr-25_11_04-2/data3159033140009759250.json', 'r') as fp:
    data = json.load(fp)

In [29]:
data.keys()

dict_keys(['type', 'crs', 'features'])

In [30]:
features = data['features']

In [31]:
len(features)

89

In [32]:
features[0].keys()

dict_keys(['type', 'geometry', 'properties', 'id'])

In [33]:
features[4]['properties']

{'feature_code': '105',
 'feature_name': 'Far West and Orana',
 'median_age_persons': 41,
 'median_tot_hhd_inc_weekly': 1106,
 'median_tot_prsnl_inc_weekly': 594,
 'sa4_code16': '105',
 'sa4_name16': 'Far West and Orana'}

In [34]:
for i in range(len(features)):
    
    print(i,'\t',features[i]['properties']['feature_name'] ,'\t',len(features[i]['geometry']['coordinates']))


0 	 Capital Region 	 109
1 	 Central Coast 	 4
2 	 Central West 	 1
3 	 Coffs Harbour - Grafton 	 26
4 	 Far West and Orana 	 1
5 	 Hunter Valley exc Newcastle 	 42
6 	 Illawarra 	 7
7 	 Mid North Coast 	 40
8 	 Murray 	 1
9 	 New England and North West 	 1
10 	 Newcastle and Lake Macquarie 	 7
11 	 Richmond - Tweed 	 9
12 	 Riverina 	 1
13 	 Southern Highlands and Shoalhaven 	 4
14 	 Sydney - Baulkham Hills and Hawkesbury 	 2
15 	 Sydney - Blacktown 	 1
16 	 Sydney - City and Inner South 	 3
17 	 Sydney - Eastern Suburbs 	 5
18 	 Sydney - Inner South West 	 1
19 	 Sydney - Inner West 	 2
20 	 Sydney - North Sydney and Hornsby 	 3
21 	 Sydney - Northern Beaches 	 3
22 	 Sydney - Outer South West 	 1
23 	 Sydney - Outer West and Blue Mountains 	 1
24 	 Sydney - Parramatta 	 1
25 	 Sydney - Ryde 	 1
26 	 Sydney - South West 	 1
27 	 Sydney - Sutherland 	 1
28 	 Ballarat 	 1
29 	 Bendigo 	 1
30 	 Geelong 	 5
31 	 Hume 	 1
32 	 Latrobe - Gippsland 	 108
33 	 Melbourne - Inner 	 1
34 	 Melb

In [35]:
from shapely.geometry import Point, Polygon

In [40]:
def geo_check(input_polygon):
    
    input_polygon = [input_polygon]
    if len(input_polygon)==1:
        pnt = Point(input_polygon[0])
        
    if len(input_polygon)>1:
        coordinate_polygon = Polygon(input_polygon)
        pnt = Point(coordinate_polygon.centriod)
        
    for i in range(len(features)):
        for j in range(len(features[i]['geometry']['coordinates'])):
     
            poly = Polygon(features[i]['geometry']['coordinates'][j][0])
            idx = int(features[i]['properties']['sa4_code16'])
            
            if pnt.within(poly):
                return {'SA4_name':features[i]['properties']['feature_name'],
                        'SA4_code':idx,
                        'GCCSA_name':mappings_df.loc[idx]['GCCSA_NAME_2016'],
                        'State_name':mappings_df.loc[idx]['STATE_NAME_2016']}
                break
            
            if poly.contains(pnt):
                return {'SA4_name':features[i]['properties']['feature_name'],
                        'SA4_code':idx,
                        'GCCSA_name':mappings_df.loc[idx]['GCCSA_NAME_2016'],
                        'State_name':mappings_df.loc[idx]['STATE_NAME_2016']}
                break
            

In [41]:
print(geo_check([151.19981289, -33.87429942]))

{'SA4_name': 'Sydney - City and Inner South', 'SA4_code': 117, 'GCCSA_name': 'Greater Sydney', 'State_name': 'New South Wales'}
